# Sentiment Analysis with Deep Learning using BERT

### Prerequisites

- Intermediate-level knowledge of Python 3 (NumPy and Pandas preferably, but not required)
- Exposure to PyTorch usage
- Basic understanding of Deep Learning and Language Models (BERT specifically)

### Project Outline

**Task 1**: Introduction (this section)

**Task 2**: Exploratory Data Analysis and Preprocessing

**Task 3**: Training/Validation Split

**Task 4**: Loading Tokenizer and Encoding our Data

**Task 5**: Setting up BERT Pretrained Model

**Task 6**: Creating Data Loaders

**Task 7**: Setting Up Optimizer and Scheduler

**Task 8**: Defining our Performance Metrics

**Task 9**: Creating our Training Loop

**Task 10**: Loading and Evaluating our Model

## Task 1: Introduction

### What is BERT

BERT is a large-scale transformer-based Language Model that can be finetuned for a variety of tasks.

For more information, the original paper can be found [here](https://arxiv.org/abs/1810.04805). 

[HuggingFace documentation](https://huggingface.co/transformers/model_doc/bert.html)

[Bert documentation](https://characters.fandom.com/wiki/Bert_(Sesame_Street) ;)

<img src="Images/BERT_diagrams.pdf" width="1000">

## Task 2: Exploratory Data Analysis and Preprocessing

We will use the SMILE Twitter dataset.

_Wang, Bo; Tsakalidis, Adam; Liakata, Maria; Zubiaga, Arkaitz; Procter, Rob; Jensen, Eric (2016): SMILE Twitter Emotion dataset. figshare. Dataset. https://doi.org/10.6084/m9.figshare.3187909.v2_

In [1]:
import torch
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
df = pd.read_csv('Data/smile-annotations-final.csv',
                names = ['id','text','category'])
# use the id column as index. The 'inplace=True' modifies the DataFrame in place without creating a new object
df.set_index('id', inplace = True)
df.head(10)

,text,category
id,,
611857364396965889,@aandraous @britishmuseum @AndrewsAntonio Merc...,nocode
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy
614877582664835073,@Sofabsports thank you for following me back. ...,happy
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy
614456889863208960,"@britishmuseum say wot, mate?",nocode
614016385442807809,Two workshops on evaluating audience engagemen...,nocode
610916556751642624,"A Forest Road, by Thomas Gainsborough 1750 Oil...",nocode


In [3]:
# Access the value from the 'text' column at the second row (index 1)
df.text.iloc[1]

'Dorian Gray with Rainbow Scarf #LoveWins (from @britishmuseum http://t.co/Q4XSwL0esu) http://t.co/h0evbTBWRq'

In [4]:
# Select rows 1 through 4 (index 1 to 4) and column 0 through 1
df.iloc[1:5, 0:1]

,text
id,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...
614877582664835073,@Sofabsports thank you for following me back. ...
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...


In [5]:
df.loc[df.index[1:5], df.columns[0]]

id
614484565059596288    Dorian Gray with Rainbow Scarf #LoveWins (from...
614746522043973632    @SelectShowcase @Tate_StIves ... Replace with ...
614877582664835073    @Sofabsports thank you for following me back. ...
611932373039644672    @britishmuseum @TudorHistory What a beautiful ...
Name: text, dtype: object

In [6]:
df.category.value_counts()

nocode               1572
happy                1137
not-relevant          214
angry                  57
surprise               35
sad                    32
happy|surprise         11
happy|sad               9
disgust|angry           7
disgust                 6
sad|disgust             2
sad|angry               2
sad|disgust|angry       1
Name: category, dtype: int64

In [7]:
# ignore the category that contains "|"
df = df[~df.category.str.contains('\|')]
df.category.value_counts()

nocode          1572
happy           1137
not-relevant     214
angry             57
surprise          35
sad               32
disgust            6
Name: category, dtype: int64

In [8]:
# ignore the category that contains 'nocode'
df = df[df.category != 'nocode']
df.category.value_counts()

happy           1137
not-relevant     214
angry             57
surprise          35
sad               32
disgust            6
Name: category, dtype: int64

In [9]:
# Map each category to a numerical representation
labels = df.category.unique()
label_dict={}
for index, label in enumerate(labels):
    label_dict[label]=index
label_dict

{'happy': 0,
 'not-relevant': 1,
 'angry': 2,
 'disgust': 3,
 'sad': 4,
 'surprise': 5}

In [10]:
# # Map the values in the 'category' column to numerical labels using the 'label_dict' dictionary and assign to a new 'label' column
df['label'] = df.category.replace(label_dict)
df.head(10)

,text,category,label
id,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0
614499696015503361,Lucky @FitzMuseum_UK! Good luck @MirandaStearn...,happy,0
613601881441570816,Yr 9 art students are off to the @britishmuseu...,happy,0
613696526297210880,@RAMMuseum Please vote for us as @sainsbury #s...,not-relevant,1
610746718641102848,#AskTheGallery Have you got plans to privatise...,not-relevant,1


## Task 3: Training/Validation Split

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
# Split the data into training and validation sets, with 15% for validation, ensuring the label distribution is preserved
X_train,X_val,y_train,y_val = train_test_split(
    df.index.values,
    df.label.values,
    test_size=0.15,
    random_state=17,
    stratify=df.label.values
)

In [13]:
# Print the first 5 values of X_train, X_val, y_train, and y_val
print("X_train (first 5):", X_train[:5])
print("X_val (first 5):", X_val[:5])
print("y_train (first 5):", y_train[:5])
print("y_val (first 5):", y_val[:5])


X_train (first 5): [614767094345936896 610755488372948992 610609791073931266
 611509879316942848 614723808407224320]
X_val (first 5): [612890300747489280 612618354612940800 612573129257394176
 614057101334347776 612364429213204482]
y_train (first 5): [0 0 0 5 0]
y_val (first 5): [0 0 0 0 0]


In [14]:
# Create a new column 'data_type' and initialize all its values to 'not_set' for every row in the DataFrame
df['data_type'] = ['not_set']*df.shape[0]

In [15]:
df.head(10)

,text,category,label,data_type
id,,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0,not_set
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0,not_set
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0,not_set
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0,not_set
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0,not_set
614499696015503361,Lucky @FitzMuseum_UK! Good luck @MirandaStearn...,happy,0,not_set
613601881441570816,Yr 9 art students are off to the @britishmuseu...,happy,0,not_set
613696526297210880,@RAMMuseum Please vote for us as @sainsbury #s...,not-relevant,1,not_set
610746718641102848,#AskTheGallery Have you got plans to privatise...,not-relevant,1,not_set


In [16]:
# Assign 'train' to the 'data_type' column for rows with indices in X_train
df.loc[X_train,'data_type'] = 'train'
# Assign 'val' to the 'data_type' column for rows with indices in X_val
df.loc[X_val,'data_type'] = 'val'

In [17]:
df.groupby(['category','label','data_type']).count()

text
category     label data_type      
angry        2     train        48
                   val           9
disgust      3     train         5
                   val           1
happy        0     train       966
                   val         171
not-relevant 1     train       182
                   val          32
sad          4     train        27
                   val           5
surprise     5     train        30
                   val           5

## Task 4: Loading Tokenizer and Encoding our Data

In [18]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [19]:
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case=True
)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [20]:
# Encode training data
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text.values,
    # Add special tokens such as [CLS] and [SEP]
    add_special_tokens=True,
    # Return attention mask
    return_attention_mask=True,
    # Pad to maximum length
    pad_to_max_length=True,
    max_length=256,
    # Explicitly enable truncation
    truncation=True,
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    truncation=True,
    return_tensors='pt'
)


In [21]:
# Filter the training data from the DataFrame
# Extract the label column
# Convert to a NumPy array
# Convert to a PyTorch tensor, PyTorch tensors are used for deep learning computations and can be processed on GPU if needed.
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type == 'train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type == 'val'].label.values)

In [22]:
dataset_train = TensorDataset(input_ids_train,
                              attention_masks_train,labels_train)
dataset_val = TensorDataset(input_ids_val,
                              attention_masks_val,labels_val)

In [23]:
len(dataset_train)

1258

In [24]:
len(dataset_val)

223

## Task 5: Setting up BERT Pretrained Model

In [25]:
from transformers import BertForSequenceClassification

In [26]:
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased', 
    num_labels = len(label_dict),
    output_attentions = False,
    output_hidden_states=False
)
    
                                      

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Task 6: Creating Data Loaders

In [27]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [28]:
batch_size = 4
dataloader_train = DataLoader(
    dataset_train,
    sampler = RandomSampler(dataset_train),
    batch_size= batch_size
)

dataloader_val = DataLoader(
    dataset_val,
    sampler = RandomSampler(dataset_val),
    batch_size= 32
)

## Task 7: Setting Up Optimizer and Scheduler

In [29]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [30]:
optimizer = AdamW(
    model.parameters(),
    # learning rate
    lr = 1e-5,
    # It is used to prevent division by zero errors during computations and ensure numerical stability.
    eps= 1e-8
)

In [31]:
epochs = 10

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps = 0,
    num_training_steps = len(dataloader_train)*epochs
)

## Task 8: Defining our Performance Metrics

Accuracy metric approach originally used in accuracy function in [this tutorial](https://mccormickml.com/2019/07/22/BERT-fine-tuning/#41-bertforsequenceclassification).

In [32]:
import numpy as np

In [33]:
from sklearn.metrics import f1_score

In [39]:
def f1_score_func(preds, labels):
    # Convert the predicted probabilities into class indices by selecting the highest value along axis 1,
    # then flatten the result into a 1D array of predicted classes.
    preds_flat = np.argmax(preds, axis = 1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average = 'weighted')

In [40]:
def accuracy_per_class(preds, labels):
    # inverse the dict
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    preds_flat = np.argmax(preds, axis = 1).flatten()
    labels_flat = labels.flatten()
    # compute the accuracy for each class
    for lebel in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat == label]
        y_true = labels_flat[labels_flat == label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds == label])}/{len(y_true)}\n')

## Task 9: Creating our Training Loop

Approach adapted from an older version of HuggingFace's `run_glue.py` script. Accessible [here](https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128).

In [41]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [42]:
# The model's weights and biases (i.e., the model's parameters) will be moved to the specified device.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)

cpu


In [43]:
def evaluate(dataloader_val):
    # In evaluation mode, Dropout is disabled, and the BatchNorm layers use the mean and variance computed during training.
    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in tqdm(dataloader_val):
        # Move each tensor in the batch to the specified device
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }
        
        # Disable gradient computation, reduce memory consumption, and improve computational efficiency.
        with torch.no_grad(): 
            # Pass the input data to the model for forward propagation and return the model's output.
            outputs = model(**inputs)
        
        # get the loss value
        loss = outputs[0]
        # get the predict value from the model 
        logits = outputs[1]
        loss_val_total += loss.item()

        # Move the predictions and true labels from the GPU to the CPU and convert them to NumPy arrays to free up GPU memory.
        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    # Axis 0 represents the row direction; when concatenating along axis 0, the number of rows in the new array will increase.
    predictions = np.concatenate(predictions, axis=0)
    # array1 = np.array([[1, 2, 3],
                   # [4, 5, 6]])
    # array2 = np.array([[7, 8, 9],
    #                    [10, 11, 12]])
    # result = np.concatenate([array1, array2], axis=0)
    # output : [[ 1  2  3]
    #  [ 4  5  6]
    #  [ 7  8  9]
    #  [10 11 12]]

    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals


In [47]:
for epoch in tqdm(range(1, epochs+1)):
    # Dropout randomly ignores some neurons in the neural network to prevent the network from overfitting to the training data.
    # BatchNorm layers calculate the mean and variance of the current batch and use these statistics to normalize the inputs. It also updates the running mean and variance.
    model.train()
    loss_train_total = 0
    # Created an iterator with a progress bar.
    progress_bar = tqdm(dataloader_train, 
                        desc='Epoch {:1d}'.format(epoch),
                        # After the progress bar iteration is complete, it will be cleared and not remain in the output.
                        leave = False,
                        # Enable progress bar
                        disable = False
                       )
    for batch in progress_bar:
        # Clears the old gradients from the previous step.
        model.zero_grad()
        # Moves each tensor in the batch to the specified device (CPU or GPU).
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1],
            'labels': batch[2]
        }
        
        outputs = model(**inputs)
        loss = outputs[0]
        loss_train_total += loss.item()
        # Computes the gradient of the loss with respect to the model parameters.
        loss.backward()
        
        # Clips the gradients to prevent exploding gradients.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        # Updates the model parameters based on the computed gradients.
        optimizer.step()
        # Updates the learning rate based on the learning rate schedule.
        scheduler.step()
        # loss.item(): Extracts the scalar value of the loss tensor, converts this tensor to a Python float.
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
        
    torch.save(model.state_dict(),f'Models/Bert_ft_epoch{epoch}.model')
    
    tqdm.write('\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader)
    tqdm.write(f'The average loss train is {loss_train_avg}')
    # Evaluate the model's performance on the validation set.
    val_loss, predicctions, true_vals = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score weighted: {val_f1}')

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/315 [00:00<?, ?it/s]

RuntimeError: File Models/Bert_ft_epoch1.model cannot be opened.

# Task 10: Loading and Evaluating our Model

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

In [ ]:
model.to(device)

In [ ]:
model.load_state_dict(
    torch.load('Model/finetuned_bert_epoch_1_gpu_trained.model',
              map_location = torch.device('cpu'))
)

In [ ]:
_, predictions,true_vals = evaluate(dataloader_val)

In [ ]:
accuracy_per_class(predictions, true_vals)